In [1]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf
tf.enable_eager_execution()

import numpy as np
import os
import time

In [2]:
# path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
# path_to_file = tf.keras.utils.get_file('iliad.txt', 'http://classics.mit.edu/Homer/iliad.mb.txt')
# path_to_file = 'C:/Users/Toby-PC/Desktop/mccarthy/'
path_to_file = 'C:/Users/Toby-PC/Desktop/got.txt'

In [3]:
# text = ''
# for i in range(5):
#     ext = "{}.txt".format(i+1)
#     path = path_to_file + ext
#     newtext = open(path, 'rb').read().decode(encoding='utf-8')
#     text += newtext
    
# text_file = open("mccarthy.txt", "w")
# text_file.write(text)
# text_file.close()

text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 7469809 characters


In [4]:
# Take a look at the first 250 characters in text
print(text[:250])

CONTENTS


COVER

TITLE PAGE



A GAME OF THRONES

A CLASH OF KINGS

A STORM OF SWORDS

A FEAST FOR CROWS

ABOUT THE AUTHOR

ALSO BY GEORGE R.R. MARTIN

EXCERPT FROM A DANCE WITH DRAGONS





A Game of Thrones

A Bantam S


In [5]:
# The unique characters in the file

vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

93 unique characters


In [6]:
print(vocab)

['\t', '\n', '\r', ' ', '!', "'", '(', ')', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '}', '©', 'é', 'ê', 'í', '–', '—', '‘', '’', '“', '”', '…']


In [7]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [8]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\t':   0,
  '\n':   1,
  '\r':   2,
  ' ' :   3,
  '!' :   4,
  "'" :   5,
  '(' :   6,
  ')' :   7,
  ',' :   8,
  '-' :   9,
  '.' :  10,
  '/' :  11,
  '0' :  12,
  '1' :  13,
  '2' :  14,
  '3' :  15,
  '4' :  16,
  '5' :  17,
  '6' :  18,
  '7' :  19,
  ...
}


In [9]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'CONTENTS\r\n\r\n\r' ---- characters mapped to int ---- > [27 39 38 44 29 38 44 43  2  1  2  1  2]


In [10]:
# The maximum length sentence we want for a single input in characters
seq_length = 500
examples_per_epoch = len(text)//seq_length

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

Instructions for updating:
Colocations handled automatically by placer.
C
O
N
T
E


In [11]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'CONTENTS\r\n\r\n\r\nCOVER\r\n\r\nTITLE PAGE\r\n\r\n\r\n\r\nA GAME OF THRONES\r\n\r\nA CLASH OF KINGS\r\n\r\nA STORM OF SWORDS\r\n\r\nA FEAST FOR CROWS\r\n\r\nABOUT THE AUTHOR\r\n\r\nALSO BY GEORGE R.R. MARTIN\r\n\r\nEXCERPT FROM A DANCE WITH DRAGONS\r\n\r\n\r\n\r\n\r\n\r\nA Game of Thrones\r\n\r\nA Bantam Spectra Book\r\n\r\nSPECTRA and the portrayal of a boxed “s” are trademarks of Bantam Books, a division of Random House, Inc.\r\n\r\nPUBLISHING HISTORY\r\n\r\nBantam hardcover edition published September 1996\r\n\r\nBantam paperback edition / September 1997\r\n\r\nMaps by'
' James Sinclair.\r\n\r\nHeraldic crests by Virginia Norey.\r\n\r\nAll rights reserved.\r\n\r\nCopyright © 1996 by George R. R. Martin\r\n\r\nLibrary of Congress Catalog Card Number: 95-43936.\r\n\r\nNo part of this book may be reproduced or transmitted in any form\r\n\r\nor by any means, electronic or mechanical, including photocopying,\r\n\r\nrecording, or by any information storage and retrieval system,\r\n\r\nw

In [12]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [13]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'CONTENTS\r\n\r\n\r\nCOVER\r\n\r\nTITLE PAGE\r\n\r\n\r\n\r\nA GAME OF THRONES\r\n\r\nA CLASH OF KINGS\r\n\r\nA STORM OF SWORDS\r\n\r\nA FEAST FOR CROWS\r\n\r\nABOUT THE AUTHOR\r\n\r\nALSO BY GEORGE R.R. MARTIN\r\n\r\nEXCERPT FROM A DANCE WITH DRAGONS\r\n\r\n\r\n\r\n\r\n\r\nA Game of Thrones\r\n\r\nA Bantam Spectra Book\r\n\r\nSPECTRA and the portrayal of a boxed “s” are trademarks of Bantam Books, a division of Random House, Inc.\r\n\r\nPUBLISHING HISTORY\r\n\r\nBantam hardcover edition published September 1996\r\n\r\nBantam paperback edition / September 1997\r\n\r\nMaps b'
Target data: 'ONTENTS\r\n\r\n\r\nCOVER\r\n\r\nTITLE PAGE\r\n\r\n\r\n\r\nA GAME OF THRONES\r\n\r\nA CLASH OF KINGS\r\n\r\nA STORM OF SWORDS\r\n\r\nA FEAST FOR CROWS\r\n\r\nABOUT THE AUTHOR\r\n\r\nALSO BY GEORGE R.R. MARTIN\r\n\r\nEXCERPT FROM A DANCE WITH DRAGONS\r\n\r\n\r\n\r\n\r\n\r\nA Game of Thrones\r\n\r\nA Bantam Spectra Book\r\n\r\nSPECTRA and the portrayal of a boxed “s” are trademarks of Bantam 

In [14]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 27 ('C')
  expected output: 39 ('O')
Step    1
  input: 39 ('O')
  expected output: 38 ('N')
Step    2
  input: 38 ('N')
  expected output: 44 ('T')
Step    3
  input: 44 ('T')
  expected output: 29 ('E')
Step    4
  input: 29 ('E')
  expected output: 38 ('N')


In [15]:
# Batch size 
BATCH_SIZE = 256
steps_per_epoch = examples_per_epoch//BATCH_SIZE

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences, 
# so it doesn't attempt to shuffle the entire sequence in memory. Instead, 
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<DatasetV1Adapter shapes: ((256, 500), (256, 500)), types: (tf.int32, tf.int32)>

In [16]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension 
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [17]:
rnn = tf.keras.layers.CuDNNGRU

In [18]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, 
                              batch_input_shape=[batch_size, None]),
    rnn(rnn_units,
        return_sequences=True, 
        recurrent_initializer='glorot_uniform',
        stateful=True),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [ ]:
model = build_model(
  vocab_size = len(vocab), 
  embedding_dim=embedding_dim, 
  rnn_units=rnn_units, 
  batch_size=BATCH_SIZE)

In [ ]:
# for input_example_batch, target_example_batch in dataset.take(1): 
#   example_batch_predictions = model(input_example_batch)
#   print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

In [ ]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (256, None, 256)          23808     
_________________________________________________________________
cu_dnngru (CuDNNGRU)         (256, None, 1024)         3938304   
_________________________________________________________________
dense (Dense)                (256, None, 93)           95325     
Total params: 4,057,437
Trainable params: 4,057,437
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
# print(sampled_indices.shape)

In [ ]:
# sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
# print(sampled_indices.shape)

In [ ]:
# sampled_indices

In [ ]:
# print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
# print()
# print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

The standard tf.keras.losses.sparse_softmax_crossentropy loss function works in this case because it is applied across the last dimension of the predictions.

In [ ]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

# example_batch_loss  = loss(target_example_batch, example_batch_predictions)
# print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)") 
# print("Target shape:      ", target_example_batch.shape)
# print("scalar_loss:      ", example_batch_loss.numpy().mean())

In [ ]:
model.compile(
    optimizer = tf.train.AdamOptimizer(),
    loss = loss)

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
EPOCHS=20

In [ ]:
history = model.fit(dataset.repeat(), epochs=EPOCHS, steps_per_epoch=steps_per_epoch, callbacks=[checkpoint_callback])

Epoch 1/20
57/58 [============================>.] - ETA: 0s - loss: 3.4229WARNING:tensorflow:From C:\Users\Toby-PC\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\network.py:1436: update_checkpoint_state (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.train.CheckpointManager to manage checkpoints rather than manually editing the Checkpoint proto.
58/58 [==============================] - 39s 679ms/step - loss: 3.4077
Epoch 2/20
58/58 [==============================] - 38s 652ms/step - loss: 2.3362
Epoch 3/20
58/58 [==============================] - 38s 653ms/step - loss: 2.1293
Epoch 4/20
58/58 [==============================] - 38s 663ms/step - loss: 1.9544
Epoch 5/20
58/58 [==============================] - 38s 659ms/step - loss: 1.7958
Epoch 6/20
58/58 [==============================] - 38s 655ms/step - loss: 1.6605
Epoch 7/20
58/58 [==============================] - 38s 662ms/

In [ ]:
tf.train.latest_checkpoint(checkpoint_dir)

In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [ ]:
model.summary()

In [ ]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing) 
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 0.5

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a multinomial distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
      
      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)
      
      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [ ]:
print(generate_text(model, start_string=u"Incandenza "))

In [ ]:
for i in range(5):
    ext = "{}.txt".format(i)
    print(ext)